In [1]:
%matplotlib notebook
import numpy as np                  # This is the standard way of importing numpy
import matplotlib.pyplot as plt     # and this is the usual way to load matplotlib
import pandas as pd                 # Pandas is convenient for tabular data

In [2]:
print("Hello")
for i in range(6):
    print(i)

Hello
0
1
2
3
4
5
